In [31]:
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
import numpy as np
import plotly.express as px
import hvplot.pandas

In [56]:
# Load the GDP data
gdp_df = pd.read_csv('..\Resources\gdp_pop_medals\gpd_medals_total.csv')
# drop unused columns
# gdp_df = gdp_df.drop(['Season','Country_Name','Gold','Silver','Bronze','Total_Medals'],1)
gdp_df = gdp_df.drop(['Season','Country_Name','Gold','Silver','Bronze'],1)
gdp_df.head()

C:\Users\Public\anaconda3\envs\mlenv\lib\site-packages\ipykernel_launcher.py:5: FutureWarning:

In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only



,Country_Code,Year,GDP,Total_Medals
0,ARG,1960,NaN,2
1,AUS,1960,1810.510249,22
2,AUT,1960,935.460427,2
3,AUT,1960,935.460427,6
4,BEL,1960,1273.691659,4


In [57]:
Country_Codes = gdp_df.Country_Code.unique()
gdp_df['cumulative'] = 0

In [58]:
for code in Country_Codes:
    gdp_df.loc[gdp_df['Country_Code'] == code,['cumulative']] = gdp_df.loc[gdp_df['Country_Code'] == code]['Total_Medals'].cumsum()

In [59]:
gdp_df.loc[gdp_df['Country_Code'] == 'ARG']['Total_Medals'].cumsum().index.values

array([  0,  38,  76, 116, 280, 324, 398, 471, 544, 617, 699, 779, 862],
      dtype=int64)

In [60]:
gdp_df

,Country_Code,Year,GDP,Total_Medals,cumulative
0,ARG,1960,NaN,2,2
1,AUS,1960,1810.510249,22,22
2,AUT,1960,935.460427,2,2
3,AUT,1960,935.460427,6,8
4,BEL,1960,1273.691659,4,4
...,...,...,...,...,...
921,UGA,2020,846.767201,4,11
922,UKR,2020,3751.740723,19,147
923,USA,2020,63530.633480,113,1826
924,UZB,2020,1749.655815,5,37


In [61]:
# Load the population data
population_df = pd.read_csv('..\Resources\gdp_pop_medals\pop_medals_total.csv')
# drop unused columns
population_df = population_df.drop(['Season','Country_Name','Gold','Silver','Bronze','Total_Medals'],1)
population_df.head()

C:\Users\Public\anaconda3\envs\mlenv\lib\site-packages\ipykernel_launcher.py:4: FutureWarning:

In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only



,Country_Code,Year,Population
0,ARG,1960,20349744.0
1,AUS,1960,10276477.0
2,AUT,1960,7047539.0
3,AUT,1960,7047539.0
4,BEL,1960,9153489.0


In [62]:
# Merge the data into one dataframe
result = []
result = pd.merge(gdp_df, population_df, how='left', on=["Country_Code", "Year"])
result

,Country_Code,Year,GDP,Total_Medals,cumulative,Population
0,ARG,1960,NaN,2,2,20349744.0
1,AUS,1960,1810.510249,22,22,10276477.0
2,AUT,1960,935.460427,2,2,7047539.0
3,AUT,1960,935.460427,2,2,7047539.0
4,AUT,1960,935.460427,6,8,7047539.0
...,...,...,...,...,...,...
1091,UGA,2020,846.767201,4,11,44404611.0
1092,UKR,2020,3751.740723,19,147,44132049.0
1093,USA,2020,63530.633480,113,1826,331501080.0
1094,UZB,2020,1749.655815,5,37,34232050.0


In [63]:
# find amount of NaN values
result.isna().sum()

Country_Code     0
Year             0
GDP             66
Total_Medals     0
cumulative       0
Population       0
dtype: int64

In [64]:
# drop NaN values in the following columns
result = result.dropna()
result

,Country_Code,Year,GDP,Total_Medals,cumulative,Population
1,AUS,1960,1810.510249,22,22,10276477.0
2,AUT,1960,935.460427,2,2,7047539.0
3,AUT,1960,935.460427,2,2,7047539.0
4,AUT,1960,935.460427,6,8,7047539.0
5,AUT,1960,935.460427,6,8,7047539.0
...,...,...,...,...,...,...
1090,TUR,2020,8561.070948,13,79,84135428.0
1091,UGA,2020,846.767201,4,11,44404611.0
1092,UKR,2020,3751.740723,19,147,44132049.0
1093,USA,2020,63530.633480,113,1826,331501080.0


In [65]:
# find amount of NaN values remaining
result.isna().sum()

Country_Code    0
Year            0
GDP             0
Total_Medals    0
cumulative      0
Population      0
dtype: int64

In [66]:
result

,Country_Code,Year,GDP,Total_Medals,cumulative,Population
1,AUS,1960,1810.510249,22,22,10276477.0
2,AUT,1960,935.460427,2,2,7047539.0
3,AUT,1960,935.460427,2,2,7047539.0
4,AUT,1960,935.460427,6,8,7047539.0
5,AUT,1960,935.460427,6,8,7047539.0
...,...,...,...,...,...,...
1090,TUR,2020,8561.070948,13,79,84135428.0
1091,UGA,2020,846.767201,4,11,44404611.0
1092,UKR,2020,3751.740723,19,147,44132049.0
1093,USA,2020,63530.633480,113,1826,331501080.0


In [67]:
result['GDP'] = result['GDP']*result['Population']
result

C:\Users\Public\anaconda3\envs\mlenv\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,Country_Code,Year,GDP,Total_Medals,cumulative,Population
1,AUS,1960,1.860567e+10,22,22,10276477.0
2,AUT,1960,6.592694e+09,2,2,7047539.0
3,AUT,1960,6.592694e+09,2,2,7047539.0
4,AUT,1960,6.592694e+09,6,8,7047539.0
5,AUT,1960,6.592694e+09,6,8,7047539.0
...,...,...,...,...,...,...
1090,TUR,2020,7.202894e+11,13,79,84135428.0
1091,UGA,2020,3.760037e+10,4,11,44404611.0
1092,UKR,2020,1.655720e+11,19,147,44132049.0
1093,USA,2020,2.106047e+13,113,1826,331501080.0


In [68]:
fig = px.scatter_3d(result, x="GDP", y="Population", z="cumulative", width=800)
fig.update_layout(legend=dict(x=0,y=1))
fig.show()

In [69]:
result.hvplot.scatter(x="GDP", y="cumulative")

:Scatter   [GDP]   (cumulative)

In [70]:
print("Number of rows:", result.shape[0])

Number of rows: 1030


In [71]:
# dropping columns unimportant to machine learning.
result = result.drop(['Year','Country_Code', 'Total_Medals'],1)

C:\Users\Public\anaconda3\envs\mlenv\lib\site-packages\ipykernel_launcher.py:2: FutureWarning:

In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only



In [72]:
# Print the names of the new columns
print(result.columns)

Index(['GDP', 'cumulative', 'Population'], dtype='object')


In [73]:
# Split the data into features and target
X = result.drop('cumulative', axis=1).values.reshape(-1, 2)
y = result['cumulative']

In [74]:
X.shape

(1030, 2)

In [75]:
model = LinearRegression()

In [76]:
model.fit(X, y)

LinearRegression()

In [77]:
y_pred = model.predict(X)
print(y_pred.shape)

(1030,)


In [78]:
# Evaluate the model
score = model.score(X, y)
print(f"R2 Score: {score}")

R2 Score: 0.7813479365373175


In [28]:
matrix = confusion_matrix(y_test, y_pred)
cm_df = pd.DataFrame(
   matrix, index=["Actual 0", "Actual 1"],
   columns=["Predicted 0", "Predicted 1"]
)
display(cm_df)

,Predicted 0,Predicted 1
Actual 0,37869,1843
Actual 1,1995,4659


In [29]:
report = classification_report(y_test, y_pred)
print(report)

              precision    recall  f1-score   support

           0       0.95      0.95      0.95     39712
           1       0.72      0.70      0.71      6654

    accuracy                           0.92     46366
   macro avg       0.83      0.83      0.83     46366
weighted avg       0.92      0.92      0.92     46366

